```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

= 分析

由题意可得：

$
Q(K, L) = a K^alpha L^beta, 0 < alpha, beta < 1
$

两边取对数，有：

$
ln Q = ln a + alpha ln K + beta ln L
$

转化为矩阵，有：

#set math.mat(delim: "[")
$
mat(ln Q)_n = mat(1, ln K, ln L)_n mat(ln a; alpha; beta)
$

记 $B=mat(ln Q)_n$，$A=mat(1, ln K, ln L)_n$，$X=mat(ln a; alpha; beta)$，则有：

$
A X = B
$

线性最小二乘法拟合：调用 `numpy.linalg.lstsq` 函数，求解 $X$ 即可。

对于非线性最小二乘法，直接把 $Q(K, L)$ 看作是 $X=mat(ln a, alpha, beta)^T$ 的非线性函数整理打包，然后调用 `scipy.optimize.leastsq` 函数，求解 $X$ 即可。

```


In [ ]:
import numpy as np
import scipy.optimize as opt

In [ ]:
# 导入数据
N = 19

Q = [
  0.7171,
  0.8964,
  1.0202,
  1.1962,
  1.4928,
  1.6909,
  1.8548,
  2.1618,
  2.6638,
  3.4634,
  4.6759,
  5.8478,
  6.7885,
  7.4463,
  7.8345,
  8.2068,
  9.9468,
  9.7315,
  10.4791,
]

K = [
  0.0910,
  0.2543,
  0.3121,
  0.3792,
  0.4754,
  0.4410,
  0.4517,
  0.5595,
  0.8080,
  1.3072,
  1.7042,
  2.0019,
  2.2914,
  2.4941,
  2.8406,
  2.9854,
  3.2918,
  3.7314,
  4.3500,
]

L = [
  4.8179,
  4.9873,
  5.1282,
  5.2783,
  5.4334,
  5.5329,
  6.4749,
  6.5491,
  6.6152,
  6.6808,
  6.7455,
  6.8065,
  6.8950,
  6.9820,
  7.0637,
  7.1394,
  7.2085,
  7.3025,
  7.3740,
]

lnQ = np.log(Q).reshape(N, 1)
lnK = np.log(K).reshape(N, 1)
lnL = np.log(L).reshape(N, 1)
A = np.hstack((np.ones((N, 1)), lnK, lnL))

In [ ]:
# 线性最小二乘法拟合
# rcond=None: 不对奇异值进行截断
X, residuals, rank, s = np.linalg.lstsq(A, lnQ, rcond=None)

# 输出结果
a, alpha, beta = np.exp(X[0, 0]), X[1, 0], X[2, 0]

print(f'a = {a}')
print(f'alpha = {alpha}')
print(f'beta = {beta}')
print(f'residuals = {residuals[0]}')

In [ ]:
# 非线性最小二乘法拟合
Q = np.array(Q).reshape(N, 1)
K = np.array(K).reshape(N, 1)
L = np.array(L).reshape(N, 1)


def func(x):
  a, alpha, beta = x[0], x[1], x[2]
  res = np.zeros(N)
  for i in range(N):
    res[i] = a * K[i, 0] ** alpha * L[i, 0] ** beta - Q[i, 0]
  return res


x0 = np.array([1, 1, 1])
x, cov = opt.leastsq(func, x0)

# 输出结果
a, alpha, beta = x[0], x[1], x[2]

print(f'a = {a}')
print(f'alpha = {alpha}')
print(f'beta = {beta}')
print(f'cov = {cov}')

```{=typst}
= 结果

== 线性最小二乘法

$
a &= 0.3148465020030749 \
alpha &= 0.6600334179383347 \
beta &= 1.2676833982522062 \
"residuals" &= 0.33733789565379535
$

== 非线性最小二乘法

$
a &= 0.833655331414795 \
alpha &= 0.7735475849339416 \
beta &= 0.7316742068690825 \
"cov" &= 1
$

= 结论

对于线性最小二乘法，$beta = 1.26 > 1$，误差较大，说明线性最小二乘法拟合效果不好。

对于非线性最小二乘法，$alpha, beta$ 二者的值均在范围内，同时 $"cov" = 1$，拟合效果较好。

其中，$alpha$ 是劳动力产出的弹性系数，$beta$ 是资本产出的弹性系数。$alpha$，$beta$ 越大，则资金，劳动力的影响越大，在资金和劳动力一定的情况下，产值越高。

```
